## Modèle ARIMA sur les contrats futures S&P500 (E-minis - ES)

Afin d'élaborer notre stratégie de trading à partir du modèle prédictif (ARIMA/GARCH) nous allons d'abord réaliser une prédiction des E-minis avec le modèle ARIMA.

Pour ce faire nous passerons par 7 étapes :

1. Le sourcing des futures S&P500 (E-minis)
2. La conception des visualisations des simulations
3. La simulation du processus stochastique ARIMA
4. Détermination du critère d'information d'Akaike (AIC)
5. Le Data fitting avec avec test de Ljung-Box
6. La vérification de la normalité avec le test de Jarque-Bera
7. La prédiction et backtesting du modèle


Commençons par importer les librairies utilisées :

In [1]:
import os
import sys

import pandas as pd
import numpy as np

import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
import statsmodels.stats as sms

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

### Le sourcing des futures S&P500 (E-minis)

Le sourcing des données porte sur les futures S&P500 (E-minis) du **18 septembre 2000 à aujourd'hui**. 

Pour faire fonctionner notre modèle nous utiliserons 

- **80 % de données d'entrainement (jusqu'au 18 septembre 2016 ~ 5000 données)** 
- **20 % de données de test (du 19 septembre 2016 à aujourd'hui ~ 1500 données)**.

Nous sourçons nos données grâce à notre fournisseur **Yahoo Finance** qui nous permet d'exploiter environ **6500 données**.

In [2]:
from backtester.dataSource.yahoo_data_source import YahooStockDataSource
startDateStr = '2000/09/18'
endDateStr = '2016/09/18'
cachedFolderName = 'yahooData/'
dataSetId = 'Futures_ES_2000_2016'
instrumentIds = ['ES=F']
ds = YahooStockDataSource(cachedFolderName=cachedFolderName,
                            dataSetId=dataSetId,
                            instrumentIds=instrumentIds,
                            startDateStr=startDateStr,
                            endDateStr=endDateStr,
                            event='history')
data = ds.getBookDataByFeature()['adjClose']

# log returns
lrets = np.log(data/data.shift(1))

Processing data for stock: ES=F
20% done...
40% done...
60% done...
80% done...


#### Pourquoi utiliser le logarithme des rendements ?

Commençons par définir un retour : $r_i$ au moment $i$, où $pi$ est le prix au moment $i$ et $j \equiv (i - 1)$ :

$$r_i = \frac {p_i - p_j}{p_j}$$
   

L'avantage de l'utilisation des ***rendements*** , par rapport aux prix, est la ***normalisation*** : mesurer toutes les variables dans une métrique comparable, permettant ainsi l'évaluation des relations analytiques entre deux ou plusieurs variables bien qu'elles proviennent de séries de prix de valeurs inégales. C'est une exigence pour de nombreuses techniques d'analyse statistique multidimensionnelle et d'apprentissage automatique. Par exemple, l'interprétation d'une matrice de covariance d'équité est rendue sensée lorsque les variables sont toutes deux mesurées en pourcentage.

Plusieurs avantages de l'utilisation des ***retours de log***, à la fois théoriques et algorithmiques.

Premièrement, la ***normalité logarithmique*** : si nous supposons que les prix sont distribués log normalement (ce qui, en pratique, peut ou non être vrai pour une série de prix donnée), alors $log(1 + r_i)$ est commodément distribué normalement, car:

$$1 + r_i = \frac{p_i}{p_j} = \exp^{\log(\frac{p_i}{p_j})}$$  

C'est pratique étant donné que la plupart des statistiques classiques présument la normalité.

Deuxièmement, ***l'égalité approximative du log brut*** : lorsque les rendements sont très faibles (courant pour les transactions avec des durées de détention courtes), l'approximation suivante garantit qu'ils sont proches en valeur des rendements bruts:

$$\log(1 + r) \approx r $$avec$$ r \ll 1$$ 

Troisièmement, ***l'additivité au temps*** : considérez une séquence ordonnée de $n$ transactions. Une statistique fréquemment calculée à partir de cette séquence est le ***rendement composé***, qui est le rendement courant de cette séquence de transactions au fil du temps :

$$\displaystyle(1 + r_1)(1 + r_2)\cdots(1 + r_n) = \prod_i(1 + r_i)$$

Cette formule est assez désagréable, car la théorie des probabilités nous le rappelle que le produit de variables normalement distribuées n'est pas normal. Au lieu de cela, la somme des variables normalement distribuées est normale (technicité importante: uniquement lorsque toutes les variables ne sont pas corrélées ), ce qui est utile lorsque l'on rappelle l'identité logarithmique suivante :

$$\log (1 + r_i) = log (\frac {p_i}{p_j}) = \log (p_i) - log (p_j)$$ 

Ainsi, les rendements composés sont normalement distribués. Enfin, cette identité nous conduit à un bénéfice algorithmique agréable; une formule simple pour calculer les rendements composés:

$$\displaystyle \sum_i \log (1 + r_i) = \log (1 + r_1) + \log (1 + r_2) + \cdots + \log (1 + r_n) = \log (p_n) - \log (p_0)$$

Ainsi, le rendement composé sur $n$ périodes est simplement la différence de log entre les périodes initiale et finale. En termes de complexité algorithmique , cette simplification réduit O (n) multiplications à O (1) additions. C'est une énorme victoire pour n modéré à grand. En outre, cette somme est utile pour les cas où les rendements divergent de la normale, car le ***théorème de la limite centrale*** nous rappelle que la moyenne de l'échantillon de cette somme convergera vers la normalité (en supposant des premier et deuxième moments finis).

Quatrièmement, la ***facilité mathématique*** : du calcul, on nous rappelle (en ignorant la constante d'intégration) :

$$e ^ x = \int e ^ x dx = \frac {d}{dx} e ^ x = e ^ x$$ 

Cette identité est extrêmement utile, car une grande partie des mathématiques financières est construite sur des ***processus stochastiques en temps continu*** qui reposent fortement sur l'intégration et la différenciation.

Cinquièmement, la stabilité numérique : l'addition de petits nombres est numériquement sûre, alors que la multiplication de petits nombres ne l'est pas car elle est sujette à un ***dépassement arithmétique***. Pour de nombreux problèmes intéressants, il s'agit d'un problème potentiel sérieux. Pour résoudre ce problème, soit l'algorithme doit être modifié pour être numériquement robuste, soit il peut être transformé en une somme numériquement sûre via des journaux.

### La conception des visualisations des simulations

### La simulation du processus stochastique ARIMA

### Détermination du critère d'information d'Akaike (AIC)

### Le Data fitting avec avec test de Ljung-Box

### La vérification de la normalité avec le test de Jarque-Bera

### La prédiction et backtesting du modèle